In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 24.4 MB/s 
     |████████████████████████████████| 452 kB 63.4 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 5.8 MB 57.0 MB/s 
     |████████████████████████████████| 1.9 MB 69.7 MB/s 
     |████████████████████████████████| 9.2 MB 68.5 MB/s 
     |████████████████████████████████| 7.6 MB 65.2 MB/s 
     |████████████████████████████████| 1.3 MB 61.5 MB/s 
     |████████████████████████████████| 182 kB 79.3 MB/s 
     |████████████████████████████████| 174 kB 78.6 MB/s 
     |████████████████████████████████| 182 kB 77.4 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 173 kB 77.5 MB/s 
     |████████████████████████████████| 168 kB 83.3 MB/s 
     |████████████████████████████████| 168 kB 85.5 MB/s 
     |█████████████████████████████

In [32]:
import pandas as pd
data = pd.read_excel('backend_data(1).xlsx')
data.head()

,word,course,sentence,tag
0,design,course 0,Sentence 0,O
1,database,course 0,Sentence 0,B-skill
2,schema,course 0,Sentence 0,O
3,using,course 0,Sentence 0,O
4,dbml,course 0,Sentence 0,B-skill


In [33]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [34]:
data["sentence"] = LabelEncoder().fit_transform(data["sentence"])
data.head()

,word,course,sentence,tag
0,design,course 0,0,O
1,database,course 0,0,B-skill
2,schema,course 0,0,O
3,using,course 0,0,O
4,dbml,course 0,0,B-skill


In [35]:
data.rename(columns={"sentence":"sentence_id","word":"words","tag":"labels"}, inplace =True)
data["labels"] = data["labels"].str.upper()
data.head()

,words,course,sentence_id,labels
0,design,course 0,0,O
1,database,course 0,0,B-SKILL
2,schema,course 0,0,O
3,using,course 0,0,O
4,dbml,course 0,0,B-SKILL


In [36]:
X = data[["sentence_id","words"]]
Y = data["labels"]

In [37]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [38]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [39]:
train_data

,sentence_id,words,labels
317,69,file,B-SKILL
804,16,database,B-SKILL
15,1,db,B-SKILL
210,51,tls,B-SKILL
944,32,section,O
...,...,...,...
534,104,project,O
453,87,sql,B-SKILL
536,105,about,O
913,30,will,O


# Model Training

In [40]:
from simpletransformers.ner import NERModel,NERArgs

In [41]:
label = data["labels"].unique().tolist()
label

['O', 'B-SKILL', 'I-SKILL']

In [42]:
args = NERArgs()
args.num_train_epochs = 6
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [43]:
model = NERModel('bert', 'bert-base-uncased', labels = label, args = args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [44]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 6:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 2 of 6:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 3 of 6:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 4 of 6:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 5 of 6:   0%|          | 0/5 [00:00<?, ?it/s]

(30, 0.3840708037217458)

In [45]:
result, model_outputs, preds_list = model.eval_model(test_data)
result

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.4206222742795944,
 'precision': 0.7111111111111111,
 'recall': 0.6037735849056604,
 'f1_score': 0.6530612244897959}

In [49]:
prediction, model_output = model.predict(["build a minimal docker image for deployment and use docker-compose for development"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'build': 'O'},
  {'a': 'O'},
  {'minimal': 'O'},
  {'docker': 'B-SKILL'},
  {'image': 'O'},
  {'for': 'O'},
  {'deployment': 'O'},
  {'and': 'O'},
  {'use': 'O'},
  {'docker-compose': 'B-SKILL'},
  {'for': 'O'},
  {'development': 'O'}]]